<p style='font-size:32px; text-align:center'> <strong>Peuplement de la base de données</strong></p>

Chargement des imports et dépendances

In [137]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [139]:
%sql sqlite:///data/orders.db3

In [141]:
import sqlite3
import csv

In [167]:
%%sql
DROP TABLE IF EXISTS customer;
CREATE TABLE customer(
    id INT NOT NULL UNIQUE,
    country VARCHAR(100) NOT NULL,
    PRIMARY KEY (id)
);    

 * sqlite:///data/orders.db3
Done.
Done.


[]

In [169]:
%%sql
DROP TABLE IF EXISTS customer_order;
CREATE TABLE customer_order(
    id INT NOT NULL UNIQUE,
    invoice_nb INT NOT NULL,
    invoice_date DATE NOT NULL,
    customer_id INT NOT NULL,
    PRIMARY KEY(id),
    CONSTRAINT FK_customer_id FOREIGN KEY (customer_id) REFERENCES customer(id)
    )    

 * sqlite:///data/orders.db3
Done.
Done.


[]

In [171]:
%%sql
DROP TABLE IF EXISTS product;
CREATE TABLE product(
    id INT NOT NULL UNIQUE,
    description VARCHAR(500) NOT NULL,
    price FLOAT NOT NULL,
    PRIMARY KEY(id)
)

 * sqlite:///data/orders.db3
Done.
Done.


[]

In [173]:
%%sql
DROP TABLE IF EXISTS order_detail;
CREATE TABLE order_detail(
    id INT NOT NULL UNIQUE,
    quantity INT NOT NULL,
    order_id INT NOT NULL,
    product_id INT NOT NULL,
    PRIMARY KEY(id),
    CONSTRAINT FK_order_id FOREIGN KEY (order_id) REFERENCES customer_order(id),
    CONSTRAINT FK_product_id FOREIGN KEY (product_id) REFERENCES product(id)
)

 * sqlite:///data/orders.db3
Done.
Done.


[]

In [350]:
'''with open('orders.csv', newline='') as orderfile : 
    order_tab = []
    lire_order = csv.DictReader(orderfile, delimiter=';')
    for ligne in lire_order:
        order_tab.append(ligne) '''

"with open('orders.csv', newline='') as orderfile : \n    order_tab = []\n    lire_order = csv.DictReader(orderfile, delimiter=';')\n    for ligne in lire_order:\n        order_tab.append(ligne) "

In [9]:
db_con = sqlite3.connect("data/orders.db3")
cur = db_con.cursor()

In [13]:
with open('orders.csv', newline='') as orderfile : 
    order_tab = []
    product = []
    customer = []
    customer_order = []
    order_detail= []
    lire_order = csv.reader(orderfile, delimiter=';')
    next(lire_order)
    
    for ligne in lire_order:
        order_tab.append(ligne)  
        customer.append([ ligne[3], ligne[4] ])
        customer_order.append([ ligne[0], ligne[1], ligne[2], ligne[3] ])
        order_detail.append([ ligne[5], ligne[6], ligne[0], ligne[7] ])
        product.append([ ligne[7], ligne[8], ligne[9] ])

In [ ]:
for prod in product:
    sql_request = f"INSERT INTO product(id, description, price) VALUES(?, ?, ?)"
    product_id = prod[0]
    description = prod[1]
    price = prod[2]

    liste_id = []
    if product_id not in liste_id:
        liste_id.append(product_id)
        
        cur.execute(sql_request, (product_id, description, price))
        db_con.commit()

In [41]:
for custom in customer:
    sql_request = f"INSERT OR IGNORE INTO customer(id, country) VALUES(?,?)"
    customer_id = custom[0]
    country = custom[1]
    cur.execute(sql_request, (customer_id, country))
    db_con.commit()

In [43]:
for cust_order in customer_order:
    sql_request = f"INSERT OR IGNORE INTO customer_order(id, invoice_nb, invoice_date, customer_id) VALUES (?, ?, ?, ?)"
    cust_order_id = cust_order[0]
    nb_inv = cust_order[1]
    date_inv = cust_order[2]
    customer_id = cust_order[3]
    cur.execute(sql_request, (cust_order_id, nb_inv, date_inv, customer_id))
    db_con.commit()

In [ ]:
for order_dt in order_detail:
    sql_request = f"INSERT OR IGNORE INTO order_detail(id, quantity, order_id, product_id) VALUES (?, ?, ?, ?)"
    cur.execute(sql_request, (order_dt[0], order_dt[1], order_dt[2], order_dt[3]))
    db_con.commit()

In [17]:
print(order_tab[0:10], "\n")
print(customers[0:10], "\n")
print(customer_orders[0:10], "\n")
print(order_details[0:10], "\n")
print(products[0:10], "\n")

[['1', '536365', '2010/12/01', '4049', 'United Kingdom', '1', '6', '756', 'GLASS STAR FROSTED T-LIGHT HOLDER', '4.25'], ['1', '536365', '2010/12/01', '4049', 'United Kingdom', '2', '2', '1606', 'SET 7 BABUSHKA NESTING BOXES', '7.65'], ['1', '536365', '2010/12/01', '4049', 'United Kingdom', '3', '6', '2650', 'WHITE METAL LANTERN', '3.39'], ['1', '536365', '2010/12/01', '4049', 'United Kingdom', '4', '6', '2803', 'RED WOOLLY HOTTIE WHITE HEART.', '3.39'], ['1', '536365', '2010/12/01', '4049', 'United Kingdom', '5', '6', '2804', 'KNITTED UNION FLAG HOT WATER BOTTLE', '3.39'], ['1', '536365', '2010/12/01', '4049', 'United Kingdom', '6', '8', '2856', 'CREAM CUPID HEARTS COAT HANGER', '2.75'], ['1', '536365', '2010/12/01', '4049', 'United Kingdom', '7', '6', '3250', 'WHITE HANGING HEART T-LIGHT HOLDER', '2.55'], ['2', '536366', '2010/12/01', '4049', 'United Kingdom', '8', '6', '1490', 'HAND WARMER RED POLKA DOT', '1.85'], ['2', '536366', '2010/12/01', '4049', 'United Kingdom', '9', '6', '149

NameError: name 'customers' is not defined

In [ ]:
db_con.close()

In [79]:
mes_tables = {'customer' : ['id', 'country'], 
              'product' : ['id', 'description', 'price'], 
              'customer_oder' : ['id', 'invoice_nb', 'invoice_date', 'customer_id'],
              'order_detail' : ['id', 'quantity', 'order_id', 'product_id']}
for table in mes_tables:
    champs = []
    for row in mes_tables[table]:
        champs.append(row)
    pt = "?,"*(len(champs)-1)+"?"
    values_ch = []
    sql_request = f"INSERT INTO {table}({champs}) VALUES ({pt})"
    print(sql_request)
    
    if table == 'customer':
        for cust in customer:
            cust.append(values_ch)
        print(values_ch)
    elif table == 'product':
        for prod in product:
            prod.append(values_ch)
        print(values_ch)
    elif table == 'customer_order':
        for cust_ord in customer_order:
            cust_ord.append(values_ch)
        print(values_ch)
    elif table == 'order_detail':
        for ord_det in order_detail:
            ord_det.append(values_ch)
        print(values_ch)

INSERT INTO customer(['id', 'country']) VALUES (?,?)
[]
INSERT INTO product(['id', 'description', 'price']) VALUES (?,?,?)
[]
INSERT INTO customer_oder(['id', 'invoice_nb', 'invoice_date', 'customer_id']) VALUES (?,?,?,?)
INSERT INTO order_detail(['id', 'quantity', 'order_id', 'product_id']) VALUES (?,?,?,?)
[]


In [175]:
db_conn = sqlite3.connect("data/orders.db3")
cursor = db_conn.cursor()

In [197]:
with open('orders.csv', newline='') as orderfile : 
    data = {'customer' : {}, 
              'product' : {}, 
              'customer_oder' : {},
              'order_detail' : {}}
    reader = csv.reader(orderfile, delimiter=';')
    next(reader)
    
    for ligne in reader :
        data['customer'][ligne[3]] = [ligne[3], ligne[4]]
        data['product'][ligne[7]] = [ ligne[7], ligne[8], ligne[9] ]
        data['customer_oder'][ligne[0]] = [ ligne[0], ligne[1], ligne[2], ligne[3] ]
        data['order_detail'][ligne[5]] =[ ligne[5], ligne[6], ligne[0], ligne[7] ]

    for table_name, table_data in data.items():
        for index, row in table_data.items():
            if table_name == 'customer':
                champs = ('id', 'country')
            elif table_name == 'product':
                champs = ('id', 'description', 'price')
            elif table_name == 'customer_order':
                champs = ('id', 'invoice_nb', 'invoice_date', 'customer_id')
            else:
                champs = ('id', 'quantity', 'order_id', 'product_id')
            sql_request = f"INSERT INTO {table_name}({champs}) VALUES ({row})"
            cursor.execute(sql_request)
            db_conn.commit()

OperationalError: near "(": syntax error

In [165]:
db_conn.close()